# Fetch concordances and analyze them

Lars G. Johnsen, National Library of Norway

lars.johnsen@nb.no

Solstrand june 2021


In this code we will fetch arbitrary concordances, and use them in conjunction with a parser.

### Initial code

In [1]:
import json
import pandas as pd
import requests
#import spacy
from dhlab.module_update import css

def search_nb(word = 'demokrati', window=20, limit = 300):
    parameters = {
        'query': word,
        'window':window,
        'limit':limit
    }
    r = requests.get("https://api.nb.no/ngram/db1/konk", params = parameters)
    return pd.DataFrame(json.loads(r.text), columns = 'doc-id concordance'.split())



The code `spacy.load` below may not work, in that case uncomment the cell below and run

In [7]:
#!python -m spacy download nb_core_news_sm

In [3]:
import spacy

nlp = spacy.load("nb_core_news_sm")

In [4]:
def parse(string):
    cols = "token.text, token.lemma_, token.pos_, token.tag_, token.dep_, token.shape_, token.is_alpha, token.is_stop".split(', ')
    doc = nlp(string)
    rows = [(token.text, token.lemma_, token.pos_, token.tag_, token.dep_,
            token.shape_, token.is_alpha, token.is_stop) for token in doc]
    return pd.DataFrame(rows, columns = cols), doc

In [5]:
pd.set_option('max_colwidth', None)
css()

# Concordances

A random collection of newspapers, periodicals and/or books are created on each query. Use SQLite fts5 syntax in formulating search https://www2.sqlite.org/fts5.html

In [8]:
%%time
df = search_nb(""" sjarken """, limit = 300)
df.style

Wall time: 716 ms


,doc-id,concordance
0,100108113,... Eller det e en av de dær latmakkan som kjærringen må jage ombord i sjarken førr å få dæm såpass...
1,200008942,"... Jeg hev fra meg skoleveska og gikk rett ombord i sjarken , sier Håvard som allerede har levd av fiske i..."
2,100122283,... Kulene slo inn i sjarken så det sprutet . Den la seg over av trykket og reiste seg igjen flere ganger...
3,100122283,"... Var det en übåt ! Nei , heldigvis , det var den blanke kula i mastetoppen på den sunkne sjarken ."
4,200101279,... Det lyktes imidlertid å få sjarken satt på land før den sank . Sjarken fikk store skader . . . . SKIEN der to 14...
5,100300588,"får sjarken i gang , Kor : O-O- U ."
6,100300588,... Men æg har solar på tanken og får sjarken i gang
7,100300588,Fer sjø ' n deinn ligg så speilblank og klukke attme ' naustet og vugge lett på sjarken som hainn Peder bøgd...
8,100300588,Men sjø ' n dein ligg lik ' speilblank og klukke attme ' naustet og vugge lett på sjarken som hainn Peder sætt...
9,100300753,"sjark en sjark , sjarken , fl. sjarker , sjarkene ( fiskebåt )"


### Copy and paste cells to run new versions

In [9]:
search_nb(""" NEAR(spiser middag, 5) """, limit = 20).style

,doc-id,concordance
0,200100007,"VI SPISER MIDDAG på en liten , hyggelig restaurant . Rettene likner ikke meget på det du er vant til . Først får..."
1,100380562,... Kanskje ei baconpølse med fransk hot dog dressing fra Esso på vei hjem og så spiser jeg middag i åtte...
2,200279769,"... Spør ikke meg ... — Jeg tror nissen spiser banan til middag ! — Nei , nissen spiser grøt , han . Og han bor på låven..."
3,200427938,"... Og uansett Kva du spiser til middag : Ikke glem grønnsakene , sier hun ."
4,200003301,"Talberg håper åpningstidene skal bidra til at pendlere kommer seg heim og spiser middag først , før de tar handleturen - og..."
5,100560222,... Etter arbeidstid spiser de middag i Folkets Hus hvor kommandoen er forlagt . Slik er de ikke tilbake her før ved...
6,200983591,"... kone en frokost , så spiser han også den , siden spiser han middag og aftens med alle de andre små spurvemenn"
7,200577193,"... mat . ­ Jeg har inntrykk av at ikke alle spiser skikkelig middag hver dag , så jeg er veldig takknemlig for den..."
8,200950532,"... Man kommer alltid til vXeilighet , enten sitter folk og spiser , - - eller de hviler middag ,"
9,201550406,... vi sitter og Spiser middag . Min hustru ser ut gjennem vinduet og roper idet hun reiser sig : « Alma ligger besvimt...


In [11]:
search_nb("NEAR(fiske* snøre*)", limit = 300).style

,doc-id,concordance
0,100022372,... Langaktig fiskesøkk av kleberstein med et hull i hver ende til snørefeste . Avbrutt gjennem hullet i den ene ende . Lengde...
1,100322718,"... brukes for å få løs fiskesnørene . Etter å ha fått opp en del snører med fin fisk , hørtes helikopterlyd som..."
2,100340901,Far reiste seg . I lommene hadde han fiskesnøre . Vi spente noen kvister og knyttet på snøret . Far var først ferdig .
3,100421943,"Patrik kjøper 10 fiskekrokar til 36,50 kr per stk. og 100 snørelodd til 1,25 kr per stk."
4,100121446,"Du trenger ikke en fiskestang for å få fisk på kroken - bare kroker , snøre og agn . Hvis du vil , kan..."
5,100121446,"... kjenne at det rykker i snøret . Hal inn fangsten , ta ut kroken og slå fisken i hodet for å drepe..."
6,100400598,... u n e e | n gong kjøpa snører som passa for dette fisket . I cpiY ] l21 ) flY QCYKHWI dlY TY...
7,100560750,... sikker på om Strickland ønsket en bekreftelse eller om han fisket . Det var på tide å skjære over snøret . Rand
8,100440817,"... Nå fiskes laks med stang og snøre , kastenot , kilenot og pålenot . 45 % av laksen tas på stang i dag , mens..."
9,100520829,"Makrell fiskes det en del av , både på garn og snøre . Det meste blir likevel tatt med landnot ."


# Parse the concordances using Spacy

### Example of `parse`

In [ ]:
parse('Han liker at hun studerer lingvistikk.')[0]

In [ ]:
parse('... Han lik att hun studerer lingvistikk.')[0]

## Collect all the concordances 

These are all contained in the variable `df`

The function parse returns the parse as a data frame or as a spacy doc

In [ ]:
# remove the phrase markers and parse the sentences

parses = [parse(s.replace("<b>", "").replace("</b>","")) for s in df['concordance']]

df_versions = [p[0] for p in parses]
spacy_docs = [p[1] for p in parses]

In [ ]:
df_parses = pd.concat(df_versions)
df_parses.head(50)

## Check out frequency of some tags

The expressions here are raw Pandas or Python. Perhaps a wrapper could be in place.

In [ ]:
df_parses.groupby("token.pos_").count()['token.text'].sort_values(ascending=False)

#### Example of wrapper for the above

In [ ]:
def count_parse(relation):
    if relation != 'token.text':
        res = df_parses.groupby(relation).count()['token.text'].sort_values(ascending=False)
    else:
        res = df_parses.groupby(relation).count()['token.dep_'].sort_values(ascending=False)
    return res

The wrapper makes it easier to write the command for inspecting the parsing result

In [ ]:
count_parse('token.dep_')

## Which words are in certain relations?

Searching using standard pandas expressions. Back to involved expressions!!

In [ ]:
df_parses[df_parses['token.dep_'] == 'nsubj'].groupby('token.text').count()['token.lemma_'].sort_values(ascending=False).head(20)

# Se på et tre

In [ ]:
df.concordance[6]

In [ ]:
spacy.displacy.render(spacy_docs[6])

# Exercise

Run the code again, and check if the frequencies change. 